# Modul 7 Persamaan Diferensial Numerik: *Nonlinear Finite Difference*, PDP Eliptik & Hiperbolik

Kembali ke [Persamaan Diferensial Numerik](./pdnum2024genap.qmd)

(work in progress!)

Melanjutkan dari modul sebelumnya, untuk masalah nilai batas PDB, kita akan membahas metode

- *Nonlinear Finite Difference*

Kemudian, kita akan membahas metode numerik untuk menyelesaikan PDP (persamaan diferensial parsial) orde 2

- yang eliptik (persamaan Laplace / persamaan Poisson); dan

- yang hiperbolik (persamaan gelombang).

## *Nonlinear Finite Difference*

Metode ini digunakan untuk mengaproksimasi masalah nilai batas:

$$
\begin{gathered}
y^{\prime \prime}=f\left(x, y, y^{\prime}\right), \quad a \leq x \leq b \\
y(a)=\alpha, \quad y(b)=\beta
\end{gathered}
$$

### *Function file* (dari *pseudocode*)

In [ ]:
function [t_grid,w]=nonlinear_FDM_pseudocode(f,f_y,f_yprime,a,b,n,alpha,beta,max_iter,TOL)
  h=(b-a)/(n+1); %sepsize
  w=zeros(n,1); %vektor solusi aproksimasi
  t_grid=[a:h:b]; %mesh_poitnya
  J=zeros(n,n); %matriks jacobian
  F=zeros(n,1); %vektor fungsi  F=(f_1,f_2,...,f_n) yang dievaluasi di x_k

  for i=1:n %inisialisasi solusi awal
    w(i)=alpha+i*(beta-alpha)/(b-a)*h;
  endfor
  k=1;
  while k<=max_iter %lakukan iterasi jika masih belum didapat kriteria stopnya

    %solve nonlinear sistem tersebut dengan metode newton
    x=a+h;
    %kontruksi matriks Jacobian, dan vektor F-nya
    t=(w(2)-alpha)/(2*h);
    J(1,1)=2+h^2*f_y(x,w(1),t); %main diagoanal
    J(1,2)=-1+(h/2)*f_yprime(x,w(1),t); %right diagonal
    F(1)=(2*w(1)-w(2)-alpha+h^2*f(x,w(1),t));
    for i =2:n-1
      x=a+i*h;
      t=(w(i+1)-w(i-1))/(2*h);
      J(i,i)=2+h^2*f_y(x,w(i),t); %main diagoanal
      J(i,i+1)=-1+(h/2)*f_yprime(x,w(i),t); %main diagoanal
      J(i,i-1)=-1-(h/2)*f_yprime(x,w(i),t); %left diagoanal
      F(i)=(2*w(i)-w(i+1)-w(i-1)+h^2*f(x,w(i),t));
    endfor
     x=b-h;
     t=(beta-w(n-1))/(2*h);
     J(n,n)=2+h^2*f_y(x,w(n),t); %main diagonal
     J(n,n-1)=-1-(h/2)*f_yprime(x,w(n),t); %right diagonal
     F(n)=(2*w(n)-w(n-1)-beta+h^2*f(x,w(n),t));



    v=inverse(J)*F; %vector v adalah product dari J^-1 F
    w= w-v; % lakukan update nilai pada w

    if norm(v,2)<= TOL %kriteria stop jika norm(v)<=toleransinya
      break;
     else
        k=k+1; %jika belum memenuhi kriteria stop terus lanjut iterasinya (memperbaiki nilai w)
    endif
  endwhile
  w=[alpha ; w ; beta]; %konstruksi akhir w
  t_grid=transpose(t_grid); % %transpose meshpoint
  % untuk konsistensi dimensi saja

endfunction

### *Function file* (dengan solusi SPL secara langsung)

Berikut ini, matriks Jacobian disusun dengan mempertimbangkan tiap elemen matriks, sehingga kompleksiasnya bisa dianggap $O(n^2)$:

In [ ]:
function [x_grid, w_grid] = nonlinear_fd_langsung_on2(f, fy, fyp, a, b, N, alph, bet, tol)
  h = (b-a) / (N+1);
  x_grid = a : h : b;
  x = x_grid(2:N+1);
  w = zeros(N, 1);
  for i = 1 : N
    w(i) = alph + i * ((bet - alph)/(b-a)) * h;
  endfor
  
  err = tol + 1;
  while !(err <= tol)
    matriks_J = zeros(N, N);
    for i = 1 : N
      for j = 1 : N
        if (i == j-1)
          if (i == 1)
            matriks_J(i, j) = -1 + h/2 * fyp(x(i), w(i), (w(i+1) - alph)/(2*h));
          elseif (i == N)
            matriks_J(i, j) = -1 + h/2 * fyp(x(i), w(i), (bet - w(i-1))/(2*h));
          else
            matriks_J(i, j) = -1 + h/2 * fyp(x(i), w(i), (w(i+1) - w(i-1))/(2*h));
          endif
        elseif (i == j)
          if (i == 1)
            matriks_J(i, j) = 2 + h^2 * fy(x(i), w(i), (w(i+1) - alph)/(2*h));
          elseif (i == N)
            matriks_J(i, j) = 2 + h^2 * fy(x(i), w(i), (bet - w(i-1))/(2*h));
          else
            matriks_J(i, j) = 2 + h^2 * fy(x(i), w(i), (w(i+1) - w(i-1))/(2*h));
          endif
        elseif (i == j+1)
          if (i == 1)
            matriks_J(i, j) = -1 - h/2 * fyp(x(i), w(i), (w(i+1) - alph)/(2*h));
          elseif (i == N)
            matriks_J(i, j) = -1 - h/2 * fyp(x(i), w(i), (bet - w(i-1))/(2*h));
          else
            matriks_J(i, j) = -1 - h/2 * fyp(x(i), w(i), (w(i+1) - w(i-1))/(2*h));
          endif
        else
          matriks_J(i, j) = 0;
        endif
      endfor
    endfor
    
    z = zeros(N, 1);
    % khusus baris pertama
    i = 1
    z(i) = -alph + 2 .* w(i) - w(i+1) + h^2 * f(x(i), w(i), (w(i+1) - alph)/(2*h));
    for i = 2 : (N-1)
      z(i) = -w(i-1) + 2 .* w(i) - w(i+1) + h^2 * f(x(i), w(i), (w(i+1) - w(i-1))/(2*h));
    endfor
    % khusus baris terakhir
    i = N;
    z(i) = -w(i-1) + 2 .* w(i) - bet + h^2 * f(x(i), w(i), (bet - w(i-1))/(2*h));
    
    % selesaikan SPL
    v = J \ z;
    
    w_baru = w + v;
    err = sum(abs(w_baru - w));
    w = w_baru;
  endwhile
endfunction

Daripada mempertimbangkan tiap elemen di matriks Jacobian, kita bisa isi matriksnya dengan nol semua terlebih dahulu, kemudian melihat per baris saja dan hanya mempertimbangkan elemen diagonal dan kolom sebelum/setelah diagonal (karena matriksnya tridiagonal).

### Contoh 1

Gunakan metode beda hingga nonlinear dengan $h=0.1$ dan toleransi $10^{-4}$ untuk mengaproksimasi BVP berikut:
$$
\begin{aligned}
y^{\prime \prime} & =y^{\prime}+2(y-\ln x)^3-\frac{1}{x}, \quad 2 \leq x \leq 3 \\
y(2) & =\frac{1}{2}+\ln 2, \quad y(3)=\frac{1}{3}+\ln 3
\end{aligned}
$$
Solusi eksak:
$$
y(x)=\frac{1}{x}+\ln x
$$

In [ ]:
f=@(x,y,yp) yp+2*(y-log(x))^3-1/x ; %fungsi f pada y=f(x,y,y')
f_y=@(x,y,yp) 6*(y-log(x))^2; %turunan fungsi f terhadap y
f_yp=@(x,y,yp) 1; %turunan fungsi f terhadap yprime
a=2; %left boundary
b=3; %right boundary
alpha=0.5+log(2); %y(a)
beta=1/3+ log(3); %y(b)
n=9; %banyaknya partisi (pilih n=9 sehingga h=0.1)
maxiter=30; %masksimal iterasi newton methodnya
TOL=10^(-4); %toleransi nilai (untuk kriteria stop)

%memanggil fungsi nonlinear_FDM_pseudocode
[x_grid,w]=nonlinear_FDM_pseudocode(f,f_y,f_yp,a,b,n,alpha,beta,maxiter,TOL)
f_anal= @(x) 1./x +log(x); %sol analitik

%membuat grafiknya
fplot(f_anal, [a,b],'b')
hold on;
scatter(x_grid,w,'r')
legend('solusi analitik', 'solusi linear FDM');
legend("location", "northwest");

%membuat tabel saja.
sol_anal=f_anal(x_grid); %sol analitik di meshpoint
error=abs(w-sol_anal); %error
[x_grid,w,sol_anal,error]

### Contoh 2

In [ ]:
f = @(x, y, yp) ((1/8)*(32 + 2 * x.^3 - y .* yp));
fy = @(x, y, yp) (-yp/8);
fyp = @(x, y, yp) (-y/8);
a = 1;
b = 3;
N = 20;
alph = 17;
bet = 43/3;
tol = 10^(-8);

[x_grid, w_grid] = nonlinear_fd_langsung(f, fy, fyp, a, b, N, alph, bet, tol);

[x_grid, w_grid]

## PDP Eliptik dengan metode Gauss-Seidel

### Bentuk umum, ide utama, penyederhanaan

### *Function file*

In [ ]:
function [x, y, w] = metode_eliptik_iteratif(lb, rb, ub, db, f, xb, xu, yb, yu, h, k, iterasi)
  dx = h;
  dy = k;
  x = xb : dx : xu;
  y = yb : dy : yu;
  n_1 = length(x);
  m_1 = length(y);
  
  % susun matriks solusi
  w = zeros(n_1, m_1);
  % mengisi syarat batas
  for i = 1 : n_1
    w(i, 1) = db(x(i), yb);
    w(i, m_1) = ub(x(i), yu);
  endfor
  for j = 2 : m_1
    w(1, j) = lb(xb, y(j));
    w(n_1, j) = rb(xu, y(j));
  endfor
  
  for iter = 1 : iterasi
    for i = 2 : n_1 - 1
      for j = 2 : m_1 - 1
        w(i, j) = w(i+1, j) + w(i-1, j) + (h/k)^2 * (w(i, j+1) + w(i, j-1));
        w(i, j) += - h^2 * f(x(i), y(j));
        w(i, j) /= 2 * ((h/k)^2 + 1);
      endfor
    endfor
  endfor
endfunction

### Contoh soal

In [ ]:
db = @(x,y) 0;
ub = @(x,y) 200*x;
lb = @(x,y) 0;
rb = @(x,y) 200*y;
f = @(x,y) 0;
xb = 0;
xu = 0.5;
yb = 0;
yu = 0.5;
dx = 0.1;
dy = 0.1;
iterasi = 50;

[x, y, w] = metode_eliptik_iteratif(db, ub, lb, rb, f, xb, xu, yb, yu, dx, dy, iterasi);

mesh(x, y, w');

## PDP Hiperbolik / Persamaan Gelombang

### Bentuk umum, ide utama, penyederhanaan

### *Function file*

### Contoh soal